# 6. Gyakrolat
![menetrend.png](images/menetrend.png)

## NQueens
![nqueen.png](images/nqueen.png)

In [ ]:
class NQueens(Problem):
    """N királynő elhelyezésének problémája egy NxN táblán úgy, hogy egyik sem üti a másikat. 
    Egy állapotot N-elemű tömbként ábrázolunk, ahol a c-edik bejegyzésben szereplő r értéke azt jelenti, hogy a 
    c oszlopban, az r sorban van egy királynő, a -1 érték pedig azt, hogy a c-edik oszlop még nem lett kitöltve. 
    Balról jobbra töltjük ki az oszlopokat.
    """

    def __init__(self, N):
        super().__init__(tuple([-1] * N))
        self.N = N

    def actions(self, state):
        """A bal szélső üres oszlopban próbálja ki az összes nem ütköző sort. """
        if state[-1] != -1:
            return []  # Minden oszlop kitöltve;
        else:
            col = state.index(-1)
            return [row for row in range(self.N)
                    if not self.conflicted(state, row, col)]

    def result(self, state, row):
        """Helyezze a következő királynőt a megadott sorba."""
        col = state.index(-1)
        new = list(state[:])
        new[col] = row
        return tuple(new)

    def conflicted(self, state, row, col):
        """Egy királynő elhelyezése (sor, oszlop) ütközik?"""
        return any(self.conflict(row, col, state[c], c)
                   for c in range(col))

    def conflict(self, row1, col1, row2, col2):
        """Összeütközésbe kerülne két királynő elhelyezése (sor1, oszlop1) és (sor2, oszlop2)?"""
        return (row1 == row2 or  # ugyanabban a sorban
                col1 == col2 or  # ugyanabban az oszlopban
                row1 - col1 == row2 - col2 or  # ugyanabban az átlóban, irány: \
                row1 + col1 == row2 + col2)  # ugyanabban az átlóban, irány: /

    def goal_test(self, state):
        """Ellenőrizze, hogy minden oszlop megtelt-e és nincs ütközés."""
        if state[-1] == -1:
            return False
        return not any(self.conflicted(state, state[col], col)
                       for col in range(len(state)))

    def h(self, node):
        """Az ütésben lévő királynők számát adja vissza egy adott csomóponthoz"""
        num_conflicts = 0
        for (r1, c1) in enumerate(node.state):
            for (r2, c2) in enumerate(node.state):
                if (r1, c1) != (r2, c2):
                    num_conflicts += self.conflict(r1, c1, r2, c2)

        return num_conflicts

## Próba hiba módszer

In [6]:
from n_queens import NQueens
from search import trial_error

nqueens = NQueens(4)
print(trial_error(nqueens))

<Node (2, -1, -1, -1)>
<Node (2, 0, -1, -1)>
<Node (2, 0, 3, -1)>
<Node (2, 0, 3, 1)>
Got it
<Node (2, 0, 3, 1)>


In [7]:
from n_queens import NQueens
from search import trial_error

nqueens = NQueens(8)
print(trial_error(nqueens))

<Node (5, -1, -1, -1, -1, -1, -1, -1)>
<Node (5, 2, -1, -1, -1, -1, -1, -1)>
<Node (5, 2, 6, -1, -1, -1, -1, -1)>
<Node (5, 2, 6, 1, -1, -1, -1, -1)>
<Node (5, 2, 6, 1, 7, -1, -1, -1)>
<Node (5, 2, 6, 1, 7, 4, -1, -1)>
<Node (5, 2, 6, 1, 7, 4, 0, -1)>
<Node (5, 2, 6, 1, 7, 4, 0, 3)>
Got it
<Node (5, 2, 6, 1, 7, 4, 0, 3)>


# A* algoritmus

In [10]:
from utils import *
from node import Node
from n_queens import NQueens

def best_first_graph_search(problem, f, display=False):
    f = memoize(f, 'f')
    # kezdő állapot létrehozása
    node = Node(problem.initial)
    # prioritásos sor létrehozása
    frontier = PriorityQueue('min', f)
    # kezdő állapot felvétele a prioritásos sorba
    frontier.append(node)
    # halmaz létrehozása a már megvizsgál elemekhez
    explored = set()

    # amíg találunk elemet
    while frontier:
        # elem kivétele a verem tetejéről
        node = frontier.pop()
        
        # ha cél állapotban vagyunk a kkor kész
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node
        
        # feldolgozott elemek bővítése
        explored.add(node.state)

        # operátorral legyártott gyermek elemek bejárása
        for child in node.expand(problem):
            # ha még nem dolgoztuk fel
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            # ha az adott pozicióbol elérhető gyermek függvény szerinti értéke
            # kisebb mint a már tárolté akkor újra rögzítjük
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None

def astar_search(problem, h=None, display=False):
    """Az A* keresés a legjobb első gráfkeresés, ahol f(n) = g(n)+h(n). Meg kell adnia a h függvényt csillagkeresés hívásakor, vagy a Probléma alosztályban."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n), display)

nqueens = NQueens(4)
print(astar_search(nqueens))

<Node (1, 3, 0, 2)>


In [2]:
from n_queens import NQueens
from search import astar_search

nqueens = NQueens(8)
print(astar_search(nqueens))

<Node (7, 1, 3, 0, 6, 4, 2, 5)>
